## **Introduction to ML for NLP [Network + Practical]**

### **LSTM**

We have seen in notebook 5.A that we can correctly train an LSTM in each language.

We now have to tune the model, and store the best results.

#### **Libraries**

We import the necessary libraries for the notebook.

In [1]:
# general
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# pytorch
import torch

# reload libs
import importlib

# custom imports
import utility.models_pytorch as CustomPytorchModule

print("> Libraries Imported")

> Libraries Imported


#### **Setup**

- We set the device to *cuda*
- We import the dataset

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("> Device:", device)

> Device: cuda


In [3]:
dataframe = pd.read_pickle("../data/3_multi_eurlex_encoded.pkl")
dataframe.head(3)

,celex_id,labels,labels_new,text_en,text_de,text_it,text_pl,text_sv,text_en_enc,text_de_enc,text_it_enc,text_pl_enc,text_sv_enc,set
0,32010D0395,2,0,commission decision of december on state aid c...,beschluss der kommission vom dezember uber die...,decisione della commissione del dicembre conce...,decyzja komisji z dnia grudnia r w sprawie pom...,kommissionens beslut av den december om det st...,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...",train
1,32012R0453,2,0,commission implementing regulation eu no of ma...,durchfuhrungsverordnung eu nr der kommission v...,regolamento di esecuzione ue n della commissio...,rozporzadzenie wykonawcze komisji ue nr z dnia...,kommissionens genomforandeforordning eu nr av ...,"[[2, 1275, 1276, 29, 100, 4, 743, 1277, 15, 12...","[[1302, 33, 1303, 3, 4, 5, 807, 15, 1304, 3, 6...","[[453, 10, 1422, 38, 14, 3, 4, 5, 990, 1423, 1...","[[1753, 1754, 3, 34, 24, 4, 5, 829, 7, 1755, 9...","[[2, 1239, 33, 23, 4, 5, 806, 7, 774, 4, 132, ...",train
2,32012D0043,2,0,commission implementing decision of january au...,durchfuhrungsbeschluss der kommission vom janu...,decisione di esecuzione della commissione del ...,decyzja wykonawcza komisji z dnia stycznia r u...,kommissionens genomforandebeslut av den januar...,"[[2, 1275, 3, 4, 1310, 1311, 15, 1015, 4, 1312...","[[1344, 3, 4, 5, 1345, 15, 1346, 74, 1347, 134...","[[2, 10, 1422, 3, 4, 5, 1454, 245, 1455, 24, 1...","[[2, 1791, 3, 4, 5, 1792, 7, 1, 1793, 1794, 65...","[[2, 1279, 4, 5, 1280, 7, 1281, 19, 1282, 1283...",train


#### **Create a Grid Search for Fine Tuning**

First, we setup the needed parameters.

In [4]:
# Parameters setup

COUNTS_EN = 3506
COUNTS_DE = 4216
COUNTS_IT = 4180
COUNTS_PL = 5255
COUNTS_SV = 4010

EMBEDDING_DIM_LIST = [1024, 2048]
HIDDEN_DIM_LIST = [1024, 2048]
LEARNING_RATE_LIST = [0.0001, 0.001]
DROPOUT_P_LIST = [0.0, 0.1]

Then, we execute a grid search.

In [5]:
for EMBEDDING_DIM in EMBEDDING_DIM_LIST:
    for HIDDEN_DIM in HIDDEN_DIM_LIST:
        for LEARNING_RATE in LEARNING_RATE_LIST:
            for DROPOUT_P in DROPOUT_P_LIST:

                # re-import the class PytorchModel
                # --> this is to be sure that each model is new, and not trained from the epoch of the previous one

                importlib.reload(CustomPytorchModule)
                PYTORCH_MODEL = CustomPytorchModule.PytorchModel

                # instantiate the model with the current parameters
                LSTM_MODEL = PYTORCH_MODEL(

                    # set model and text language
                    model_type      = "LSTM_fixed",
                    dataset         = dataframe,
                    language        = "en",

                    # set device, bacth size and epochs
                    device          = device,
                    batch_size      = 64,
                    epochs          = 50,

                    # set hyperparameters
                    vocab_size      = COUNTS_EN,
                    embedding_dim   = EMBEDDING_DIM,
                    hidden_dim      = HIDDEN_DIM,
                    learning_rate   = LEARNING_RATE,
                    dropout_p       = DROPOUT_P
                )

                # train the model
                LSTM_MODEL.train_model()

> Parameters imported
> Dataset correctly divided in training set, validation set and test set
> Created Pytorch datasets and dataloaders
> Model 'LSTM_fixed' instantiated
> Initialization required 1.3459 seconds
> Training Started
  - Total Epochs: 50


> Epoch 1: 100%|██████████| 60/60 [00:33<00:00,  1.79it/s]


 - Training Loss        1.0987
 - Validation Loss      1.0987
 - Validation Accuracy  0.3448

 - Validation Accuracy (per class)
   * Class 0	 0.0855 [29 out of 339]
   * Class 1	 0.9169 [287 out of 313]
   * Class 2	 0.0487 [15 out of 308]
   * Mean        0.3504

> ATTENTION: epoch 1 was the best one so far! The model has been saved :)



> Epoch 2: 100%|██████████| 60/60 [00:32<00:00,  1.83it/s]


 - Training Loss        1.0708
 - Validation Loss      1.0991
 - Validation Accuracy  0.3448

 - Validation Accuracy (per class)
   * Class 0	 0.0413 [14 out of 339]
   * Class 1	 0.9105 [285 out of 313]
   * Class 2	 0.1039 [32 out of 308]
   * Mean        0.3519

> ATTENTION: epoch 2 was the best one so far! The model has been saved :)



...

 - Training Loss        0.0243
 - Validation Loss      0.7665
 - Validation Accuracy  0.8292

 - Validation Accuracy (per class)
   * Class 0	 0.7935 [269 out of 339]
   * Class 1	 0.8147 [255 out of 313]
   * Class 2	 0.8831 [272 out of 308]
   * Mean        0.8304



> Epoch 49: 100%|██████████| 60/60 [01:42<00:00,  1.71s/it]


 - Training Loss        0.022
 - Validation Loss      0.8329
 - Validation Accuracy  0.8333

 - Validation Accuracy (per class)
   * Class 0	 0.7817 [265 out of 339]
   * Class 1	 0.8339 [261 out of 313]
   * Class 2	 0.8896 [274 out of 308]
   * Mean        0.8351



> Epoch 50: 100%|██████████| 60/60 [01:42<00:00,  1.71s/it]


 - Training Loss        0.0213
 - Validation Loss      0.8188
 - Validation Accuracy  0.8313

 - Validation Accuracy (per class)
   * Class 0	 0.7817 [265 out of 339]
   * Class 1	 0.8371 [262 out of 313]
   * Class 2	 0.8799 [271 out of 308]
   * Mean        0.8329

